Introduction
------------

The `ebcc` package implements various coupled cluster (CC) models for both purely electronic and coupled electron-boson models, with a focus on generality and model extensibility.

The solvers are built on top of the mean-field classes of [PySCF](https://github.com/pyscf/pyscf.git), and behave in a somewhat similar fashion to the post-mean-field methods of PySCF.

Calculations in `ebcc` proceed by constructing the molecule and running a self-consistent-field calculation using PySCF.

In [1]:
import numpy as np
from pyscf import gto, scf

mol = gto.Mole()
mol.atom = "H 0 0 0; F 0 0 1.1"
mol.basis = "aug-cc-pvdz"
mol.verbose = 4
mol.build()

mf = scf.RHF(mol)
mf.kernel()

System: uname_result(system='Linux', node='ollie-desktop', release='6.8.0-49-generic', version='#49~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Nov  6 17:42:15 UTC 2', machine='x86_64')  Threads 1
Python 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
numpy 1.26.1  scipy 1.14.0  h5py 3.10.0
Date: Sat Nov 23 15:25:41 2024
PySCF version 2.6.2
PySCF path  /home/ollie/.local/lib/python3.10/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr   0.0
[INPUT]  2 F      0.000000000000   0.000000000000   1.100000000000 AA    0.000000000000   0.000000000

-100.00114790766195

`ebcc` uses the default python `Logger` class. There is a default one that logs to `stderr`, but we'll configure one here that better suits the tutorial. Logging can be disabled by using an appropriate custom logger, using `ebcc.core.logging.NullLogger`, or by setting the level of `ebcc.default_log` to be very high.

In [2]:
import sys
from logging import StreamHandler
from ebcc.core.logging import Logger

log = Logger("main")
log.setLevel(0)
log.addHandler(StreamHandler(sys.stdout))

Many ansatzes under the umbrella of the coupled cluster (CC) method are supported, using generating code with intermediate optimisation and spin integration. The ansatz of choice can be controlled using the `ansatz` keyword when initialising a calculation.

Just like PySCF methods, the calculation can be performed using the `kernel` method, which will attempt to converge a ground state.

In [3]:
from ebcc import EBCC

ccsd = EBCC(mf, ansatz="CCSD", log=log)
ccsd.kernel()

print(f"Correlation energy: {ccsd.e_corr:.10f}")
print(f"Total energy: {ccsd.e_tot:.10f}")

        _
       | |
   ___ | |__    ___   ___
  / _ \| '_ \  / __| / __|
 |  __/| |_) || (__ | (__
  \___||_.__/  \___| \___|
                     1.5.0
numpy:
 > Version:  1.26.1
 > Git hash: N/A
pyscf:
 > Version:  2.6.2
 > Git hash: N/A
ebcc:
 > Version:  1.5.0
 > Git hash: N/A
OMP_NUM_THREADS = 1


RCCSD
*****

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD

Space: (5o, 27v)

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.2348416212    -100.2359895289
   1    -0.2307958451    -100.2319437528     4.046e-03     1.889e-02
   2    -0.2367214447    -100.2378693524     5.926e-03     5.030e-03
   3    -0.2365839704    -100.2377318780     1.375e-04     2.655e-03
   4    -0.2374320912    -100.2385799989     8.481e-04     1.458e-03
   5    -0.2374369120    -100.2385848196     4.821e-06     1.090e-04
   6    -0.2374237335    -100.2385716412

A similar solver is available for the $\Lambda$ amplitudes, where the chosen ansatz supports them. Users should note that if one wishes to calculate the density matrices, before solving the $\Lambda$ equations, the approximation $\Lambda = T^\dagger$ will be used, leading to approximate density matrices.

In [4]:
rdm1_approx = ccsd.make_rdm1_f()

ccsd.solve_lambda()

rdm1 = ccsd.make_rdm1_f()

print(f"Error in approximate 1RDM: {np.max(np.abs(rdm1_approx - rdm1)):.3g}")

Solving for de-excitation (lambda) amplitudes.

Iter      Δ(Ampl.)
   1     8.501e-03
   2     1.246e-03
   3     9.129e-04
   4     5.651e-04
   5     3.060e-05
   6     1.289e-05
   7     4.416e-06
   8     1.261e-06
   9     3.142e-07
  10     1.614e-07
  11     7.844e-08
  12     1.510e-08
  13     2.888e-09

Converged.

Time elapsed: 287 ms


Error in approximate 1RDM: 0.00842


After converging the ground state, some ansatzes offer equation-of-motion (EOM) operations, and `ebcc` has a solver to converge them via the Davidson method. Note that the default logging behaviour is that the EOM solver inherits the logger passed to the ground state.

In [5]:
eom = ccsd.ip_eom(nroots=5)
eom.kernel()

print(f"First ionisation potential: {-eom.e[0]:.8f}")
print(f"Singles weight: {np.linalg.norm(eom.v[:, 0][:ccsd.nocc])**2:.4g}")


IP-EOM-RCCSD
************

Options:
 > nroots:  5
 > e_tol:  1e-06
 > max_iter:  100
 > max_space:  12

Solving for IP excitations using the Davidson solver.

Converged.

Root           Energy        Weight    Conv.
   0     0.5573504810       0.95419     True
   1     0.5573553537       0.95419     True
   2     0.6780342604       0.96114     True
   3     1.3574272581    0.00053347     True
   4     1.3695781840        0.6841     True

Time elapsed: 176 ms

First ionisation potential: -0.55735048
Singles weight: 0.9542


Brueckner orbital self-consistency is also generally available to methods. The `brueckner` method automatically calls the `kernel` itself this time, updating in-place the coefficients and amplitudes. For finer control the solvers can be used from `ebcc.opt`.

In [6]:
ccsd.brueckner()

print(f"T1 norm: {np.linalg.norm(ccsd.t1):.3g}")


RBCCSD
Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  20
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Solving for Brueckner orbitals.

Iter   Energy (corr.)      Energy (tot.)    Conv.     Δ(Energy)          |T1|
   0    -0.2374225572    -100.2385704648     True
   1    -0.2405424771    -100.2377076113     True     3.120e-03     7.248e-03
   2    -0.2396041405    -100.2378515978     True     1.440e-04     1.930e-03
   3    -0.2397546740    -100.2378203372     True     3.126e-05     1.479e-05
   4    -0.2397542965    -100.2378205140     True     1.767e-07     4.722e-07
   5    -0.2397542967    -100.2378205119     True     2.028e-09     2.912e-08
   6    -0.2397542975    -100.2378205119     True     4.846e-12     6.940e-09

Converged.

E(corr) = -0.2397542975
E(tot)  = -100.2378205119

Time elapsed: 818 ms

T1 norm: 6.94e-09


Other canonical and approximate ansatzes are available. To list the supported ansatzes, there is a helper method `available_models`. For more details on which methods are available for each ansatz, the `FEATURES.md` file contains a summary.

In [9]:
from ebcc import available_models

_ = available_models()

RHF:
  RCC2, RCC3, RCCD, RCCSD, RCCSD(T), RCCSDT, RCCSD_SD_1_1, RCCSD_SD_1_2, RCCSD_S_1_1, RCCSDt, RCCSDt, RDCD, RDCSD, RDFCC2, RDFCCD, RDFCCSD, RDFDCD, RDFDCSD, RDFQCISD, RMP2, RMP3, RQCISD
UHF:
  UCC2, UCC3, UCCD, UCCSD, UCCSD(T), UCCSDT, UCCSD_SD_1_1, UCCSD_SD_1_2, UCCSD_S_1_1, UCCSDt, UCCSDt, UDCD, UDCSD, UDFCC2, UDFCCD, UDFCCSD, UDFDCD, UDFDCSD, UDFQCISD, UMP2, UMP3, UQCISD
GHF:
  GCC2, GCC3, GCCD, GCCSD, GCCSD(T), GCCSDT, GCCSDTQ, GCCSD_SD_1_1, GCCSD_SD_1_2, GCCSD_S_1_1, GCCSDt, GCCSDt, GMP2, GMP3, GQCISD


There is some support for methods requiring a perturbative correction to the ground state energy, which is automatically calculated with the ground state `kernel`.

In [11]:
from ebcc import REBCC

ccsd_t = REBCC(mf, ansatz="CCSD(T)", log=log)


RCCSD(T)
********

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD(T)

Space: (5o, 27v)

